<a href="https://colab.research.google.com/github/RazerRaymond/Pricing-Simulations/blob/main/delta_hedge_unfixedvol.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Cost of delta hedging in BMS Model with non-constant volatility (Call)

In [ ]:
import numpy as np
from scipy.stats import norm
def BMS_d1(S, K, r, q, sigma, tau):
    ''' Computes d1 for the Black Merton Scholes formula '''
    d1 = 1.0*(np.log(1.0 * S/K) + (r - q + sigma**2/2) * tau) / (sigma * 
np.sqrt(tau))
    return d1
def BMS_d2(S, K, r, q, sigma, tau):
    ''' Computes d2 for the Black Merton Scholes formula '''
    d2 = 1.0*(np.log(1.0 * S/K) + (r - q - sigma**2/2) * tau) / (sigma * 
np.sqrt(tau))
    return d2
def BMS_price(type_option, S, K, r, q, sigma, T, t=0):
    ''' Computes the Black Merton Scholes price for a 'call' or 'put' option '''
    tau = T - t
    d1 = BMS_d1(S, K, r, q, sigma, tau)
    d2 = BMS_d2(S, K, r, q, sigma, tau)
    if type_option == 'call':
        price = S * np.exp(-q * tau) * norm.cdf(d1) - K * np.exp(-r * tau) * norm.cdf(d2)
    elif type_option == 'put':
        price = K * np.exp(-r * tau) * norm.cdf(-d2) - S * np.exp(-q * tau) * norm.cdf(-d1) 
    return price
def BMS_delta(type_option, S, K, r, q, sigma, T, t=0):
    ''' Computes the delta for a call or a put. '''
    tau = T - t
    d1 = BMS_d1(S, K, r, q, sigma, tau)
    if type_option == 'call':
        delta = np.exp(-q * tau) * norm.cdf(d1)
    elif type_option == 'put':
        delta = np.exp(-q * tau) * (norm.cdf(d1) - 1)
    return delta

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from time import time

# problem data
spot = 100
K = 110
r = 0
q = 0
sig = 0.22

maturity = 1

m = 365
dt = maturity/m

# ir and divr
r_d = r / m
q_d = q / m


In [ ]:
n_sim = 100
np.random.seed(46542150)
synthetic_C = np.zeros(n_sim)

st = time()

for j in range(n_sim):
    
    # reset for each path
    S = spot
    T = maturity
    delta_C_prev = 0.0
    sig_i = sig

    for i in range(m):
        sig_i = sig - 0.001 * (S - spot)
        delta_C = BMS_delta('call', S, K, r, q, sig, T)
        # Take derivative to calculate new delta cr: yuyang xu
        delta_C -= 0.001 * S * norm.pdf(BMS_d1(S, K, r, q, sig_i, T)) * np.sqrt(T)
        synthetic_C[j] += (delta_C - delta_C_prev)*S
        # Note - net position is always delta_c
    
        delta_C_prev = delta_C
        
        z = np.random.randn()
        S = S * np.exp((r - q - sig_i*sig_i/2)*dt + sig_i*np.sqrt(dt)*z)
        
        T = T - dt
    
    synthetic_C[j] += (-delta_C*S + np.maximum(S-K,0))

et = time()
print('Elapsed time is %s seconds.' % str(et - st))

Elapsed time is 9.75707221031189 seconds.


In [ ]:
print('mean_synthetic_C with active volatility = ' + str(np.mean(synthetic_C)))
print('std_synthetic_C with active volatility = ' + str(np.std(synthetic_C, ddof=1)))

mean_synthetic_C with active volatility = 4.912587127941605
std_synthetic_C with active volatility = 0.5650661421289541
